In [14]:
from astropy import units as u
from astropy.io import fits
import numpy as np
from astropy.nddata import CCDData
from subprocess import Popen
import ccdproc
from astropy.modeling import models

In [15]:
#OVERSCANNNN
#bias_files = [line.rstrip('\n') for line in open('/Users/kyleporter/Desktop/ASTR480/Wolf1346/all_allfiles.list')]
#for file in bias_files:
#    data = fits.getdata("/Users/kyleporter/Desktop/ASTR480/RAW_Wolf1346/" + file)
#    new_data = np.delete(data, slice(1023,1026), 0)
#    newer_data = np.delete(new_data, slice(1026,1077), 1)
#    fits.writeto('/Users/kyleporter/Desktop/ASTR480/Wolf_overscan_cut/' + file, newer_data)

In [16]:
bias_files = [line.rstrip('\n') for line in open('/Users/kyleporter/Desktop/ASTR480/Wolf1346/all_biases.txt')]
data_stack = []
for file in bias_files:
    data_stack.append(fits.getdata("/Users/kyleporter/Desktop/ASTR480/Wolf_overscan_cut/" + file))

In [17]:
medianBias = np.median(data_stack,axis=0) 
medianBias
header = fits.getheader("/Users/kyleporter/Desktop/ASTR480/Wolf_overscan_cut/" + bias_files[0])
header['HISTORY'] = 'Median combined'
header
fits.getdata("/Users/kyleporter/Desktop/ASTR480/Wolf_overscan_cut/" + file)
#fits.writeto('/Users/kyleporter/Desktop/ASTR480/Wolf_overscan_cut/master_bias.fits',medianBias,header)

array([[ 318,  320, 1507, ..., 1086,  337,  339],
       [ 304,  303,  333, ...,  341,  312,  310],
       [ 312,  311,  340, ...,  348,  319,  316],
       ...,
       [ 320,  320,  348, ...,  349,  321,  319],
       [ 308,  304,  334, ...,  341,  304,  307],
       [ 325,  325, 1324, ..., 1654,  334,  333]], dtype=uint16)

In [23]:
medianBias

array([[ 318.,  316., 1531., ..., 1086.,  339.,  338.],
       [ 304.,  302.,  333., ...,  341.,  309.,  310.],
       [ 314.,  311.,  342., ...,  350.,  317.,  316.],
       ...,
       [ 318.,  316.,  348., ...,  350.,  318.,  319.],
       [ 307.,  306.,  335., ...,  341.,  307.,  309.],
       [ 327.,  327., 1316., ..., 1654.,  333.,  334.]])

In [8]:
data_stack

[array([[ 320,  316, 1564, ..., 1104,  340,  340],
        [ 304,  301,  337, ...,  340,  308,  310],
        [ 315,  316,  338, ...,  349,  314,  319],
        ...,
        [ 320,  313,  342, ...,  349,  317,  317],
        [ 310,  306,  336, ...,  342,  307,  311],
        [ 329,  327, 1318, ..., 1676,  333,  333]], dtype=uint16),
 array([[ 318,  315, 1624, ..., 1170,  335,  337],
        [ 302,  301,  330, ...,  341,  309,  307],
        [ 313,  308,  342, ...,  348,  317,  316],
        ...,
        [ 319,  316,  344, ...,  347,  318,  320],
        [ 303,  303,  338, ...,  343,  309,  309],
        [ 326,  327, 1332, ..., 1612,  334,  334]], dtype=uint16),
 array([[ 318,  316, 1506, ..., 1062,  339,  343],
        [ 300,  302,  332, ...,  343,  309,  311],
        [ 312,  312,  344, ...,  352,  316,  316],
        ...,
        [ 317,  317,  350, ...,  350,  322,  321],
        [ 307,  306,  339, ...,  341,  310,  306],
        [ 330,  329, 1299, ..., 1660,  337,  334]], dtype=uint

In [26]:
make_no_bias_files = [line.rstrip('\n') for line in open('/Users/kyleporter/Desktop/ASTR480/Wolf1346/all_allfiles.list')]
data_stack = []

n = len(bias_files)

for i in range(0,n):
    data,header = fits.getdata('/Users/kyleporter/Desktop/ASTR480/Wolf_overscan_cut/'+make_no_bias_files[i],header=True)
    dataout = data - medianBias
    header['HISTORY'] = 'Bias subtracted'
    fits.writeto("/Users/kyleporter/Desktop/ASTR480/Wolf_no_bias/no_bias_" + make_no_bias_files[i], dataout, header)

In [27]:
# flat_headers = []
# flat_files = [line.rstrip('\n') for line in open('/Users/kyleporter/Desktop/ASTR480/Wolf1346/all_flats.txt')]
# for i in flat_files:
#     flat_headers.append(fits.getheader("/Users/kyleporter/Desktop/ASTR480/Wolf1346/" + i)["FILTER"])
#     flat_headers.append(i)

In [31]:
flatfiles_washm = [line.rstrip('\n') for line in open('/Users/kyleporter/Desktop/ASTR480/Wolf1346/all_wash.list')]
flatfiles_JCRC = [line.rstrip('\n') for line in open('/Users/kyleporter/Desktop/ASTR480/Wolf1346/all_JCRC.list')]

washm_flat_stack = []
for file in flatfiles_washm:
    data,header = fits.getdata('/Users/kyleporter/Desktop/ASTR480/Wolf1346/'+file,header=True)
    data = data / np.median(data)
    washm_flat_stack.append(data)

washm_flat=np.median(washm_flat_stack,axis=0)
m=np.mean(washm_flat)
washm_flat_avg=washm_flat/m

header['HISTORY'] = 'Combined and normalized flat field'
fits.writeto('/Users/kyleporter/Desktop/ASTR480/Wolf_flatted/washm_flat_avg.fits',washm_flat_avg,header)

JCRC_flat_stack = []
for file in flatfiles_JCRC:
    data,header = fits.getdata('/Users/kyleporter/Desktop/ASTR480/Wolf1346/'+file,header=True)
    data = data / np.median(data)
    JCRC_flat_stack.append(data)
    
JCRC_flat=np.median(JCRC_flat_stack,axis=0)
m=np.mean(JCRC_flat)
JCRC_flat_avg=JCRC_flat/m

header['HISTORY'] = 'Combined and normalized flat field'
fits.writeto('/Users/kyleporter/Desktop/ASTR480/Wolf_flatted/JCRC_flat_avg.fits',JCRC_flat_avg,header)

n=len(washm_datain)

for i in range(0,n):
    data,header = fits.getdata(washm_datain[i],header=True)
    dataout = data / washm_flat_avg
    header['HISTORY'] = 'Flat Fielded'
    fits.writeto(washm_dataout[i],dataout,header)

    
n2=len(JCRC_datain)

for i in range(0,n2):
    data,header = fits.getdata(JCRC_datain[i],header=True)
    dataout = data / JCRC_flat_avg
    header['HISTORY'] = 'Flat Fielded'
    fits.writeto(JCRC_dataout[i],dataout,header)


VerifyError: 
Verification reported errors:
HDU 0:
    Card 35:
        Card 'CD1_1' is not FITS standard (invalid value string: '+NAN                 / WCS (1/InstScaleX)*cos(InstAng)').
    Card 36:
        Card 'CD1_2' is not FITS standard (invalid value string: '+NAN                 / WCS (1/InstScaleY)*sin(InstAng)').
    Card 37:
        Card 'CD2_1' is not FITS standard (invalid value string: '+NAN                 / WCS (-1/(InstScaleX)*sin(InstAng)').
    Card 38:
        Card 'CD2_2' is not FITS standard (invalid value string: '+NAN                 / WCS (1/InstScaleY)*cos(InstAng)').
Note: astropy.io.fits uses zero-based indexing.
